In [1]:
%pip install langchain==0.3.23 | tail -n 1 
%pip install langchain-ibm==0.3.10 | tail -n 1 
%pip install langchain-community==0.3.16 | tail -n 1 
%pip install wikipedia==1.4.0 | tail -n 1
%pip install openai==1.77.0 | tail -n 1
%pip install langchain-openai==0.3.16 | tail -n 1
%pip install langgraph==0.6.1 | tail -n 1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_ibm import ChatWatsonx
from langchain.agents import AgentType


In [3]:
llm = ChatWatsonx(
    model_id="ibm/granite-3-2-8b-instruct",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="skills-network",
)

In [4]:
response = llm.invoke("What framework do you recommend for building AI Agents - LangChain or LangGraph?")
print("\nResponse Content: ", response.content)


Response Content:  Both LangChain and LangGraph are interesting projects in the AI domain, but neither is typically used as a primary framework for building AI agents. 

1. LangChain: This is a decentralized application (dApp) platform that enables the creation of language-based apps, not specifically an AI agent framework. It's designed for building decentralized applications that incorporate language models and blockchain technologies, focusing more on the infrastructure and interaction rather than the core AI agent development.

2. LangGraph: LangGraph is a knowledge graph construction system from semantic web data sources that uses language models to accelerate the process. It serves as a tool to automatically build a knowledge graph from unstructured data, utilizing natural language processing, but again, it's not a framework for building AI agents per se.

For building AI agents, you might want to consider more established frameworks such as:

1. TensorFlow or PyTorch: These are

In [5]:
from langgraph.prebuilt import create_react_agent

In [6]:
from langchain_core.tools import tool
import re

In [7]:
@tool
def sum_numbers_from_text(inputs: str) -> float:
    """
    Adds a list of numbers provided in the input string.
    
    Args:
        text: A string containing numbers that should be extracted and summed.
        
    Returns:
        The sum of all numbers found in the input.
    """
    # Use regular expressions to extract all numbers from the input
    numbers = [int(num) for num in re.findall(r'\d+', inputs)]
    result = sum(numbers)
    return result

In [8]:
print("Name: \n", sum_numbers_from_text.name)
print("Description: \n", sum_numbers_from_text.description) 
print("Args: \n", sum_numbers_from_text.args) 

Name: 
 sum_numbers_from_text
Description: 
 Adds a list of numbers provided in the input string.

Args:
    text: A string containing numbers that should be extracted and summed.

Returns:
    The sum of all numbers found in the input.
Args: 
 {'inputs': {'title': 'Inputs', 'type': 'string'}}


In [9]:
agent_exec = create_react_agent(model=llm, tools=[sum_numbers_from_text])
msgs = agent_exec.invoke({"messages": [("human", "Add the numbers -10, -20, -30")]})

In [10]:
print(msgs["messages"][-1].content)

The sum of the numbers -10, -20, and -30 is 60.


In [ ]:
from langchain_community.utilities import WikipediaAPIWrapper

# Create a Wikipedia tool using the @tool decorator
@tool
def search_wikipedia(query: str) -> str:
    """Search Wikipedia for factual information about a topic.
    
    Parameters:
    - query (str): The topic or question to search for on Wikipedia
    
    Returns:
    - str: A summary of relevant information from Wikipedia
    """
    wiki = WikipediaAPIWrapper()
    return wiki.run(query)

In [ ]:
search_wikipedia.invoke("What is tool calling?")

In [ ]:
# Update your tools list to include the Wikipedia tool
tools_updated = [sum_numbers_from_text, search_wikipedia]

# Create the agent with all tools including Wikipedia
math_agent_updated = create_react_agent(
    model=llm,
    tools=tools_updated,
    prompt="You are a helpful assistant that can perform various mathematical operations and look up information. Use the tools precisely and explain your reasoning clearly."
)

In [ ]:
query = "What is the population of Canada? Add another million to it"

response = math_agent_updated.invoke({"messages": [("human", query)]})

print("\nMessage sequence:")
for i, msg in enumerate(response["messages"]):
    print(f"\n--- Message {i+1} ---")
    print(f"Type: {type(msg).__name__}")
    if hasattr(msg, 'content'):
        print(f"Content: {msg.content}")
    if hasattr(msg, 'name'):
        print(f"Name: {msg.name}")
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"Tool calls: {msg.tool_calls}")